In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import os
from datetime import timedelta
from kloppy import skillcorner


# Setup pitch and plot
from mplsoccer import Pitch
#from mplsoccer.pitch import Pitch ,VerticalPitch

# username = "XXX"
# password = "XXX"


# from skillcorner.client import SkillcornerClient
# client=SkillcornerClient(username=username,password=password)

def time_to_seconds(time_str):
    if time_str is None:
        return 90 * 60  # 120 minutes = 7200 seconds
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

In [ ]:
# CELL TO LOAD ALL MATCHES TRACKING DATA, PLAYERS DATA AND EVENTS DATA

matches_json_path = os.path.join(os.getcwd(), "data/matches.json")

with open(matches_json_path, "r") as f:
    matches_json = json.load(f)

match_ids = [match["id"] for match in matches_json]

all_tracking_dfs = []

for match_id in match_ids:
    tracking_data_github_url = f'https://media.githubusercontent.com/media/SkillCorner/opendata/741bdb798b0c1835057e3fa77244c1571a00e4aa/data/matches/{match_id}/{match_id}_tracking_extrapolated.jsonl'
    
    try:
        raw_data = pd.read_json(tracking_data_github_url, lines=True)
        
        raw_df = pd.json_normalize(
            raw_data.to_dict("records"),
            "player_data",
            ["frame", "timestamp", "period", "possession", "ball_data"],
        )
        
        # Extract 'player_id' and 'group' from possession
        raw_df["possession_player_id"] = raw_df["possession"].apply(lambda x: x.get("player_id"))
        raw_df["possession_group"] = raw_df["possession"].apply(lambda x: x.get("group"))
        
        # Expand ball_data
        raw_df[["ball_x", "ball_y", "ball_z", "is_detected_ball"]] = pd.json_normalize(raw_df.ball_data)
        
        # Drop unnecessary columns
        raw_df = raw_df.drop(columns=["possession", "ball_data"])
        
        # Add match_id
        raw_df["match_id"] = match_id
        
        all_tracking_dfs.append(raw_df)
        
    except Exception as e:
        print(f"Failed to load match {match_id}: {e}")

tracking_df_all = pd.concat(all_tracking_dfs, ignore_index=True)
tracking_df_all.head()

# -------------------------------------------------------------------------------------

all_players_dfs = []

for match_id in match_ids:
    meta_data_github_url = f'https://raw.githubusercontent.com/SkillCorner/opendata/741bdb798b0c1835057e3fa77244c1571a00e4aa/data/matches/{match_id}/{match_id}_match.json'
    
    try:
        response = requests.get(meta_data_github_url)
        raw_match_data = response.json()
        
        raw_match_df = pd.json_normalize(raw_match_data, max_level=2)
        raw_match_df["home_team_side"] = raw_match_df["home_team_side"].astype(str)
        
        players_df = pd.json_normalize(
            raw_match_df.to_dict("records"),
            record_path="players",
            meta=[
                "home_team_score",
                "away_team_score",
                "date_time",
                "home_team_side",
                "home_team.name",
                "home_team.id",
                "away_team.name",
                "away_team.id",
            ],
        )
        
        # Keep only players who played
        players_df = players_df[~((players_df.start_time.isna()) & (players_df.end_time.isna()))]
        players_df["total_time"] = players_df["end_time"].apply(time_to_seconds) - players_df["start_time"].apply(time_to_seconds)
        players_df["is_gk"] = players_df["player_role.acronym"] == "GK"
        players_df["match_name"] = players_df["home_team.name"] + " vs " + players_df["away_team.name"]
        players_df["home_away_player"] = np.where(players_df.team_id == players_df["home_team.id"], "Home", "Away")
        players_df["team_name"] = np.where(players_df.team_id == players_df["home_team.id"], players_df["home_team.name"], players_df["away_team.name"])
        
        # Figure out sides
        players_df[["home_team_side_1st_half", "home_team_side_2nd_half"]] = (
            players_df["home_team_side"].astype(str).str.strip("[]").str.replace("'", "").str.split(", ", expand=True)
        )
        players_df["direction_player_1st_half"] = np.where(players_df.home_away_player == "Home",
                                                           players_df.home_team_side_1st_half,
                                                           players_df.home_team_side_2nd_half)
        players_df["direction_player_2nd_half"] = np.where(players_df.home_away_player == "Home",
                                                           players_df.home_team_side_2nd_half,
                                                           players_df.home_team_side_1st_half)
                                                           
        # Keep only relevant columns
        columns_to_keep = [
            "start_time",
            "end_time",
            "match_name",
            "date_time",
            "home_team.name",
            "away_team.name",
            "id",
            "short_name",
            "number",
            "team_id",
            "team_name",
            "player_role.position_group",
            "total_time",
            "player_role.name",
            "player_role.acronym",
            "is_gk",
            "direction_player_1st_half",
            "direction_player_2nd_half",
            "playing_time.total.minutes_played",
        ]
        players_df = players_df[columns_to_keep]
        
        all_players_dfs.append(players_df)
        
    except Exception as e:
        print(f"Failed to process match {match_id}: {e}")

all_players_df = pd.concat(all_players_dfs, ignore_index=True)
all_players_df.head()
all_players_df.shape

# -------------------------------------------------------------------------------------

# Merging datasets
enriched_all_tracking_data = tracking_df_all.merge(
    all_players_df, left_on=["player_id"], right_on=["id"]
)
enriched_all_tracking_data.head()

# -------------------------------------------------------------------------------------

all_de_dfs = []

for match_id in match_ids:
    url = f"https://raw.githubusercontent.com/SkillCorner/opendata/master/data/matches/{match_id}/{match_id}_dynamic_events.csv"
    try:
        de_match = pd.read_csv(url)
        all_de_dfs.append(de_match)
    except Exception as e:
        print(f"Failed to load dynamic events for match {match_id}: {e}")

de_all_matches = pd.concat(all_de_dfs, ignore_index=True)
print(de_all_matches.shape)

